In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from pathlib import Path
import json
import pandas as pd

# Step 0: detect project root (folder that has "data")
ROOT_DIR = Path.cwd()
for _ in range(6):
    if (ROOT_DIR / "data").exists():
        break
    ROOT_DIR = ROOT_DIR.parent

print("Detected project root:", ROOT_DIR)

raw_excel_path = ROOT_DIR / "data" / "raw" / "synthetic" / "B2B_Customer_Feedback_Dataset.xlsx"
processed_dir = ROOT_DIR / "data" / "processed" / "summarising"
processed_dir.mkdir(parents=True, exist_ok=True)
processed_jsonl_path = processed_dir / "summarising_input.jsonl"

print("Raw Excel:", raw_excel_path)
print("Processed JSONL:", processed_jsonl_path)

if not raw_excel_path.exists():
    raise FileNotFoundError(f"Raw Excel not found: {raw_excel_path}")

Detected project root: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing
Raw Excel: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\data\raw\synthetic\B2B_Customer_Feedback_Dataset.xlsx
Processed JSONL: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\data\processed\summarising\summarising_input.jsonl


In [15]:
# Load raw Excel
df_raw = pd.read_excel(raw_excel_path)
print("Rows in raw Excel:", len(df_raw))
df_raw.head()

Rows in raw Excel: 50


,Comment_ID,Comment
0,1,Status?
1,2,As per our call just now pls rush the 6 inch A...
2,3,"MTC received but cert date shows March 2024, o..."
3,4,Can faster or not? Client side keep asking me ...
4,5,Good support from your Katherine during LKG pr...


In [16]:
# Normalise columns to Comment_ID / Comment
lower_map = {c.lower(): c for c in df_raw.columns}
id_col = lower_map.get("comment_id") or lower_map.get("id")
text_col = lower_map.get("comment") or lower_map.get("text") or lower_map.get("message")

if id_col is None or text_col is None:
    raise ValueError(
        f"Could not find Comment_ID/Comment columns in {raw_excel_path}. "
        f"Found columns: {list(df_raw.columns)}"
    )

df_proc = df_raw.rename(columns={id_col: "Comment_ID", text_col: "Comment"})
df_proc = df_proc[["Comment_ID", "Comment"]].copy()

# Save as JSONL for the pipeline
with processed_jsonl_path.open("w", encoding="utf-8") as f:
    for _, row in df_proc.iterrows():
        rec = {
            "Comment_ID": int(row["Comment_ID"]) if pd.notna(row["Comment_ID"]) else None,
            "Comment": str(row["Comment"]) if pd.notna(row["Comment"]) else "",
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("✅ Saved processed input JSONL:", processed_jsonl_path)
df_proc.head()

✅ Saved processed input JSONL: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\data\processed\summarising\summarising_input.jsonl


,Comment_ID,Comment
0,1,Status?
1,2,As per our call just now pls rush the 6 inch A...
2,3,"MTC received but cert date shows March 2024, o..."
3,4,Can faster or not? Client side keep asking me ...
4,5,Good support from your Katherine during LKG pr...


In [17]:
import importlib
import sys

# Make sure Python can find notebooks/summarising/cel
CEL_DIR = ROOT_DIR / "notebooks" / "summarising" / "cel"
if str(CEL_DIR) not in sys.path:
    sys.path.append(str(CEL_DIR))

import pipeline
importlib.reload(pipeline)  # reload in case you edit it

results = pipeline.run_summarising_pipeline(
    input_path=processed_jsonl_path,
    output_dir=CEL_DIR / "results",
    test_mode=True,      # change to False for full run
    max_rows=30,
    model="gpt-4.1-mini",
    sleep_sec=0.0,
)

results

📥 Loading processed data from: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\data\processed\summarising\summarising_input.jsonl
Rows in dataset: 50
TEST_MODE=True → processing first 30 rows only

--- Row 0 (Comment_ID=1) ---
Input: Status?
JSON output: {'improvement_comment': 'Provide clearer status updates to customers.'}

--- Row 1 (Comment_ID=2) ---
Input: As per our call just now pls rush the 6 inch ANSI 150 flanges to Tuas site by 3pm today. Foreman waiting.
JSON output: {'improvement_comment': 'Improve delivery speed to ensure 6 inch ANSI 150 flanges reach Tuas site by 3pm.'}

--- Row 2 (Comment_ID=3) ---
Input: MTC received but cert date shows March 2024, our PO is Feb 2024. Which batch is this? Pls clarify asap.
JSON output: {'improvement_comment': 'Provide clear batch date information matching the PO date for better clarity.'}

--- Row 3 (Comment_ID=4) ---
Input: Can faster or not? Client side keep asking me already. Need the DI fittings by COB toda

{'full': WindowsPath('c:/Users/tengc/Downloads/develop_ai_pipelines_testing/ai_pipeline_testing/notebooks/summarising/cel/results/cx_improvement_full.jsonl'),
 'only': WindowsPath('c:/Users/tengc/Downloads/develop_ai_pipelines_testing/ai_pipeline_testing/notebooks/summarising/cel/results/cx_improvement_only.jsonl')}

In [18]:
results_dir = ROOT_DIR / "notebooks" / "summarising" / "cel" / "results"
full_path = results_dir / "cx_improvement_full.jsonl"
only_path = results_dir / "cx_improvement_only.jsonl"

print("Full path:", full_path)
print("Only path:", only_path)

def load_jsonl(path: Path) -> pd.DataFrame:
    records = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            records.append(json.loads(line))
    return pd.DataFrame(records)

df_full = load_jsonl(full_path)
df_only = load_jsonl(only_path)

print("Full rows:", len(df_full))
print("Improvement-only rows:", len(df_only))
df_full.head()

Full path: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\notebooks\summarising\cel\results\cx_improvement_full.jsonl
Only path: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\notebooks\summarising\cel\results\cx_improvement_only.jsonl
Full rows: 30
Improvement-only rows: 28


,Comment_ID,Comment,model_output
0,1,Status?,{'improvement_comment': 'Provide clearer statu...
1,2,As per our call just now pls rush the 6 inch A...,{'improvement_comment': 'Improve delivery spee...
2,3,"MTC received but cert date shows March 2024, o...",{'improvement_comment': 'Provide clear batch d...
3,4,Can faster or not? Client side keep asking me ...,{'improvement_comment': 'Please expedite the d...
4,5,Good support from your Katherine during LKG pr...,{'improvement_comment': 'Maintain the same ser...


In [19]:
def categorize(c):
    if c == "NONE":
        return "NONE"
    if c == "ERROR":
        return "ERROR"
    if not isinstance(c, str) or not c.strip():
        return "EMPTY"
    return "IMPROVEMENT"

df_full["category"] = df_full["model_output"].apply(
    lambda mo: categorize(mo.get("improvement_comment"))
)

df_full["category"].value_counts()

category
IMPROVEMENT    28
NONE            2
Name: count, dtype: int64

In [20]:
sample_good = df_full[df_full["category"] == "IMPROVEMENT"].head(10).copy()
sample_good["improvement_comment"] = sample_good["model_output"].apply(
    lambda mo: mo.get("improvement_comment", "")
)
sample_good[["Comment_ID", "Comment", "improvement_comment"]]

,Comment_ID,Comment,improvement_comment
0,1,Status?,Provide clearer status updates to customers.
1,2,As per our call just now pls rush the 6 inch A...,Improve delivery speed to ensure 6 inch ANSI 1...
2,3,"MTC received but cert date shows March 2024, o...",Provide clear batch date information matching ...
3,4,Can faster or not? Client side keep asking me ...,Please expedite the delivery of DI fittings to...
4,5,Good support from your Katherine during LKG pr...,Maintain the same service level for upcoming J...
5,6,Wrong item sent again. This is 3rd time alread...,Improve order accuracy to ensure the correct p...
6,7,Refer to my email dated 15 Oct regarding the m...,Respond promptly to customer emails regarding ...
7,8,Driver cannot find our Pioneer Road location. ...,Provide clearer location details or add a cont...
8,9,Hi the valve you quoted is for freshwater syst...,Provide valves that are corrosion-resistant fo...
10,11,Boss asking for update on PO CC-2847. Can advi...,Provide an estimated time of arrival for PO CC...


In [21]:
sample_bad = df_full[df_full["category"].isin(["NONE", "ERROR"])].head(10).copy()
sample_bad["improvement_comment"] = sample_bad["model_output"].apply(
    lambda mo: mo.get("improvement_comment", "")
)
sample_bad[["Comment_ID", "Comment", "category", "improvement_comment"]]

,Comment_ID,Comment,category,improvement_comment
9,10,"Flanges received yesterday, quality looks good...",NONE,NONE
18,19,Thanks for rushing out the emergency order las...,NONE,NONE


In [22]:
flat = df_full.copy()
flat["improvement_comment"] = flat["model_output"].apply(
    lambda mo: mo.get("improvement_comment", "")
)
flat = flat[["Comment_ID", "Comment", "improvement_comment"]]

csv_path = results_dir / "cx_improvement_flat.csv"
flat.to_csv(csv_path, index=False, encoding="utf-8-sig")
csv_path

WindowsPath('c:/Users/tengc/Downloads/develop_ai_pipelines_testing/ai_pipeline_testing/notebooks/summarising/cel/results/cx_improvement_flat.csv')

In [23]:
import sys, importlib, json
from pathlib import Path

# Make sure Python can find notebooks/summarising/cel
CEL_DIR = ROOT_DIR / "notebooks" / "summarising" / "cel"
if str(CEL_DIR) not in sys.path:
    sys.path.append(str(CEL_DIR))

import postprocessing
importlib.reload(postprocessing)

# df_full currently has nested model_output; flatten improvement_comment column
df_agg = df_full.copy()
df_agg["improvement_comment"] = df_agg["model_output"].apply(
    lambda mo: mo.get("improvement_comment", "")
)

agg_save_path = results_dir / "cx_improvement_aggregate.json"

agg = postprocessing.aggregate_from_df(
    df_agg,
    improvement_col="improvement_comment",
    model="gpt-4.1-mini",
    max_items=None,                 # or e.g. 120 if you want a cap
    save_path=agg_save_path,
)

print("Saved aggregated summary to:", agg_save_path)
print(json.dumps(agg, indent=2, ensure_ascii=False))

Saved aggregated summary to: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\notebooks\summarising\cel\results\cx_improvement_aggregate.json
{
  "summary_overall": [
    "Customers demand faster, clearer, and more reliable communication across all interactions.",
    "Delivery speed and accuracy are critical to avoid project delays and maintain customer trust.",
    "Product specification clarity and quality assurance are essential to meet application requirements and avoid rework."
  ],
  "categories": {
    "Product": [
      {
        "theme": "Specification Clarity",
        "issue_summary": "Product mismatches and unclear specifications cause project delays and rework.",
        "action": "Establish earlier, clearer communication channels with clients to confirm product specs before order processing."
      },
      {
        "theme": "Quality and Certification",
        "issue_summary": "Customers face issues with product quality such as coating defects 